In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import datetime
import numpy as np
import tensorflow as tf
from architecture import build_unet
from pipeline import collect_image_paths, tf_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

In [2]:
input_files_csv = "all_files_harmonized.csv"
csv_path = "data.csv"
batch_size = 2
input_shape = (256, 256, 3)
epochs = 1
lr = 1e-4
model_path = "results/"

In [3]:
images_vv = collect_image_paths(input_files_csv, img_type="vv")
masks_fld = collect_image_paths(input_files_csv, img_type="flood_label")

In [4]:
images_vv[123], masks_fld[123]

('data/train/bangladesh_20170314t115609/tiles/vv/bangladesh_20170314t115609_x-11_y-41_vv.png',
 'data/train/bangladesh_20170314t115609/tiles/flood_label/bangladesh_20170314t115609_x-11_y-41.png')

In [5]:
# train test split
train_x, test_x = train_test_split(images_vv, test_size=0.2, random_state=112)
train_y, test_y = train_test_split(masks_fld, test_size=0.2, random_state=112)

In [6]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 26724 - 26724
Test: 6681 - 6681


In [7]:
train_dataset = tf_dataset(train_x[:1000], train_y[:1000], batch=batch_size)
test_dataset = tf_dataset(test_x[:200], test_y[:200], batch=batch_size)

In [8]:
model = build_unet(input_shape)
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision()
    ]
)

In [9]:
# model.summary()

In [10]:
ckpt_path = os.path.join(model_path, str(datetime.datetime.now()).replace(" ", "_"), "model.h5")
callbacks = [
        ModelCheckpoint(ckpt_path, monitor="val_loss", verbose=1),
        ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor="val_loss", patience=10)
    ]

In [11]:
train_steps = len(train_x)//batch_size
if len(train_x) % batch_size != 0:
    train_steps += 1

test_steps = len(test_x)//batch_size
if len(test_x) % batch_size != 0:
    test_steps += 1
    
print(f"train_steps: {train_steps}")
print(f"test_steps: {test_steps}")

train_steps: 13362
test_steps: 3341


In [ ]:
model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=test_steps,
        callbacks=callbacks
    )

   10/13362 [..............................] - ETA: 24:25:46 - loss: 0.5792 - mean_io_u: 0.4905 - recall: 0.2155 - precision: 0.0148